In [ ]:
pip install tensorflow

In [173]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [213]:
df = pd.read_csv("../input/primaindiansdiabetes/pima-indians-diabetes.csv", header = None)
df.head()

In [214]:
# set columns names
df.columns = ['Pregnancies', 'Gluucose', 'BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction', 'Age', 'Outcome']
df

In [215]:
df.shape

In [216]:
df[df['Outcome'] == 1].count()

In [217]:
df[df['Outcome'] == 0].count()

In [218]:
df.isnull().sum()

In [219]:
import seaborn as sns

In [220]:
sns.pairplot(df, hue="Outcome")

In [221]:
from sklearn.model_selection import train_test_split

### independent features x1,x2,x3,x4 like that
X = df.drop('Outcome', axis = 1).values
### dependent feature y
y = df['Outcome'].values

## train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state = 0)

In [222]:
### starting use of pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
### creating tensors

X_train =  torch.FloatTensor(X_train)
X_test =  torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [231]:
### creating Modelwith PyTorch
class ANN_Model(nn.Module):
    def __init__(self,input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.out=nn.Linear(hidden2,out_features)
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=self.out(x)
        return x

In [232]:
### instantiate my_model
torch.manual_seed(20)
model = ANN_Model()

In [233]:
model.parameters

In [236]:
### Backward Propagation -- Define the loss_function, define the optimizer 
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [237]:
epochs=500
final_losses=[]
for i in range(epochs):
    i=i+1
    y_pred=model.forward(X_train)
    loss=loss_function(y_pred,y_train)
    final_losses.append(loss)
    if i%100==1:
        print("Epoch number: {} and the loss : {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [238]:
### plot the loss function
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
final_losses[:5]

In [ ]:
finalLoss = []
for i in range(len(final_losses)):
    f = final_losses[i].item()
    finalLoss.append(f)

In [ ]:
plt.plot(range(epochs),finalLoss)
plt.ylabel('Loss')
plt.xlabel('Epoch')

In [ ]:
#### Prediction In X_test data
predictions=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_pred=model(data)
        print(y_pred)
        predictions.append(y_pred.argmax().item())
#         print(y_pred.argmax().item())

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,predictions)
cm

In [ ]:
plt.figure(figsize=(5,3))
sns.heatmap(cm,annot=True)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score,matthews_corrcoef
score=accuracy_score(y_test,predictions)
f1 = f1_score(y_test, predictions)
mcc = matthews_corrcoef(y_test, predictions)
print(f"accuracy {score} f1 {f1} mcc {mcc}")

### Dataset is imbalanced. That's why when I flipped the classes, f1 score changed. To avoid such situations, we don't need to run to the code from beginning. we can do it by calculating MCC. But I am repeating the whole process once again

In [197]:
### Matthew's Correlation Coefficient (MCC)
import numpy as np
df['Outcome']=np.where(df['Outcome']==1,0,1)
df.head()

In [198]:
from sklearn.model_selection import train_test_split

### independent features x1,x2,x3,x4 like that
X = df.drop('Outcome', axis = 1).values
### dependent feature y
y = df['Outcome'].values

## train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state = 0)

In [199]:
### starting use of pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
### creating tensors

X_train =  torch.FloatTensor(X_train)
X_test =  torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [200]:
### creating Modelwith PyTorch
class ANN_Model(nn.Module):
    def __init__(self,input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.out=nn.Linear(hidden2,out_features)
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=self.out(x)
        return x

In [201]:
### instantiate my_model
torch.manual_seed(20)
model = ANN_Model()

In [202]:
### Backward Propagation -- Define the loss_function, define the optimizer 
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [203]:
epochs=500
final_losses=[]
for i in range(epochs):
    i=i+1
    y_pred=model.forward(X_train)
    loss=loss_function(y_pred,y_train)
    final_losses.append(loss)
    if i%100==1:
        print("Epoch number: {} and the loss : {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [204]:
### plot the loss function
import matplotlib.pyplot as plt
%matplotlib inline

In [205]:
finalLoss = []
for i in range(len(final_losses)):
    f = final_losses[i].item()
    finalLoss.append(f)

In [206]:
plt.plot(range(epochs),finalLoss)
plt.ylabel('Loss')
plt.xlabel('Epoch')

In [207]:
#### Prediction In X_test data
predictions=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_pred=model(data)
        print(y_pred)
        predictions.append(y_pred.argmax().item())
#         print(y_pred.argmax().item())

In [208]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,predictions)
cm

In [209]:
plt.figure(figsize=(10,6))
sns.heatmap(cm,annot=True)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

In [210]:
from sklearn.metrics import accuracy_score, f1_score,matthews_corrcoef
score=accuracy_score(y_test,predictions)
f1 = f1_score(y_test, predictions)
mcc = matthews_corrcoef(y_test, predictions)
print(f"accuracy {score} f1 {f1} mcc {mcc}")

In [239]:
#### Save the model
torch.save(model,'diabetes.pt')

In [240]:
#### Save And Load the model
model=torch.load('diabetes.pt')

In [241]:
model.eval()

In [242]:
### Predcition of new data point
list(df.iloc[0,:-1])

In [243]:
#### New Data
lst1=[6.0, 130.0, 72.0, 40.0, 0.0, 25.6, 0.627, 45.0]

In [244]:
new_data=torch.tensor(lst1)

In [245]:
#### Predict new data using Pytorch
with torch.no_grad():
    print(model(new_data))
    print(model(new_data).argmax().item())